# Experiment Env. for Signal DIE Prototype CTM on RHSC-ET Thermal Analysis #

In [1]:
#### import packages ####
import os
import ipywidgets as widgets
from ipywidgets import Label, HBox, VBox
from ipyfilechooser import FileChooser

#### import MY packages ####
from parsers import RHSCETparser
from setups import FCpowermap
from setups import utilities

In [2]:
### Create CASE ###
CASE = widgets.Text(value="TEST", description="CASE Name:", disabled=False)
PWfc = FileChooser("./")   ### power file generated from FC
MDfc = FileChooser("./")
TFfc = FileChooser("./")
PWType = widgets.RadioButtons(options=["IPF","CSV","TILE"], description="Power file type", disabled=False)
HB1 = HBox([Label("Select Power File"), PWfc, PWType])
HB2 = HBox([Label("Select Metal Density File"), MDfc])
HB3 = HBox([Label("Select Totem TechFile"), TFfc])
VBox([CASE, HB1, HB2, HB3])
#display(PWfc)

In [3]:
### Create case in CASEs folder base on CASE Name ###
#import importlib
#importlib.reload(utilities)
root = os.path.join(os.getcwd(), "CASEs")
caseFolder = utilities.createCASE(root, CASE.value)
caseDB = os.path.join(caseFolder, "DB")

In [4]:
### Optional: Manually given the design area ###
isManually = widgets.Select(options=["True", "False"], value="False", description="Design Area in Manually", disabled=False)
llx = widgets.FloatText(value=0.0, description="LLX", disabled=False)
lly = widgets.FloatText(value=0.0, description="LLY", disabled=False)
urx = widgets.FloatText(value=0.0, description="URX", disabled=False)
ury = widgets.FloatText(value=0.0, description="URY", disabled=False)
HB1 = HBox([llx, lly, urx, ury])
VBox([isManually, HB1])

In [5]:
### Exam Power Map ###
import importlib
importlib.reload(FCpowermap)

designArea = []
try:
    if isManually.value in ["True"]:
        designArea = [llx.value, lly.value, urx.value, ury.value]
except:
    pass

print("Design Area: {}".format(designArea))
FCpwrView = FCpowermap.FCPowerView(PWfc.selected, PWType.value, DesignArea=designArea)

#FCpwrView.modifyCSV(modifyCells=["ihscrb000ab1h06x5", "ihsctb000ab1h36x5"], pwrFactor=100.0)

topCells = FCpwrView.getTopNHighPD()
topCells = ["z_intf[1].hbi_intf_0/g106733","z_intf[0].hbi_intf_0/prects_FE_RC_661_0","z_intf[0].hbi_intf_0/g101670",\
            "z_intf[0].hbi_intf_0/prects_FE_RC_1024_0","z_intf[1].hbi_intf_0/g103393","g1070597,g1mmbn022aa1d36x5",\
            "z_intf[1].hbi_intf_0/g106584","z_intf[1].hbi_intf_0/g103395","z_intf[0].hbi_intf_0/prects_FE_RC_641_0"]
print(topCells)
FCpwrView.plot(highlighted=topCells, isShow=False)

Design Area: []
<INFO> Instances Area:0.275,0.16,215.775,207.2; Total PWR=5.038mW
['z_intf[1].hbi_intf_0/g106733', 'z_intf[0].hbi_intf_0/prects_FE_RC_661_0', 'z_intf[0].hbi_intf_0/g101670', 'z_intf[0].hbi_intf_0/prects_FE_RC_1024_0', 'z_intf[1].hbi_intf_0/g103393', 'g1070597,g1mmbn022aa1d36x5', 'z_intf[1].hbi_intf_0/g106584', 'z_intf[1].hbi_intf_0/g103395', 'z_intf[0].hbi_intf_0/prects_FE_RC_641_0']


In [24]:
#### Optional, in-test: setup metal density ####
from setups import MetalDensity
import importlib
importlib.reload(FCpowermap)
importlib.reload(MetalDensity)

resolution = 1
FCpwrView = FCpowermap.FCPowerView(PWfc.selected, PWType.value, DesignArea=designArea)
FCpwrView.translate2TilePwr(resolution=resolution)

tilePWR = FCpwrView.tileDict[resolution]

MetalDensity.MetalDensityScheme(tilePWR, scheme=["WORST"])
#FCpwrView.plot(ptype=["TILE", resolution], saveImg="TilePwr_{}um.png".format(str(resolution)))
#FCpwrView.TilePwrAnalysis(resolution, topN=5)

<INFO> Instances Area:0.325,0.96,215.575,206.4; Total PWR=1.667mW


In [25]:
#### Optional: setup ET metal density input file ####
MDPath = "./Metal_density_setup"
MetalDensity.loadMetalDensitySetup(TFfc.selected, MDPath, area=designArea)

C:\Users\chienchi\OneDrive - Intel Corporation\Documents\PyCodes\Thermal\fc_powermap\fake.tech Parsing...
[['FEOL'], ['MEOL'], ['M0'], ['VIA0'], ['M1'], ['VIA1'], ['M2'], ['VIA2'], ['M3'], ['VIA3'], ['M4'], ['VIA4'], ['M5'], ['VIA5'], ['M6'], ['VIA6'], ['M7'], ['VIA7'], ['M8'], ['VIA8'], ['M9'], ['VIA9'], ['M10']]


In [ ]:
### Create Prototype CTM ###
designArea, ETmhsPath = FCpwrView.RHSCETmhs(outputFolder=caseDB)
CTM_name = widgets.Text(value="CTM01", description="CTM Name", disabled=False)
CTM_resolution = widgets.IntText(value=1, description="CTM Resolution", disabled=False)
HB1 = HBox([CTM_resolution, Label("um")])
VBox([CTM_name, HB1])

In [ ]:
#print(designArea, PWfc.selected, MDfc.selected, TFfc.selected, CTM_resolution.value)
DIE_X = "{:.3f}".format(designArea[2]-designArea[0])
DIE_Y = "{:.3f}".format(designArea[3]-designArea[1])
CTM_r = str(CTM_resolution.value)

### make a copy to the case DB folder ###
os.system("cp -rf {} {}".format(PWfc.selected, caseDB))
os.system("cp -rf {} {}".format(MDfc.selected, caseDB))
os.system("cp -rf {} {}".format(TFfc.selected, caseDB))
CTMparaDict = {"RUN_DIR":caseFolder, "CTMNAME":CTM_name.value, "DIE_LENGTH":DIE_X, "DIE_WIDTH":DIE_Y, \
               "RESOLUTION":CTM_r, "PWRMAP":ETmhsPath, "DENSITY":MDfc.selected, "TOTEM_TF":TFfc.selected}

templateScript = os.path.join(os.path.join(os.getcwd(), "ET_scripts"), "prototypeCTM.tcl")
ETscript = utilities.createETscript(CTMparaDict, template=templateScript, outputFolder=caseDB, outputName="genPrototypeCTM.tcl")

runcmd = ["#!/bin/csh", "rm -rf {}".format(os.path.join(caseFolder, "*CTM*")),\
          "source /nfs/site/disks/ifs_itip_disk001/cchuang/Tools/licenses/ansys.csh", \
          "redhawk_sc_et -3dic -ng {}".format(ETscript)]
runcshPath = os.path.join(caseDB, "genCTM.csh")

progBar = widgets.IntProgress(max=100)
display(progBar)
CTMpath = os.path.join(os.path.join(CTMparaDict["RUN_DIR"], "CTM"), CTMparaDict["CTMNAME"]+".tar.gz")
utilities.submitJob(runcmd, runcshPath=runcshPath, progressBar=progBar, checkFiles=[CTMpath])

In [ ]:
### Exam CTM power profile ###
#ctmImgPath = os.path.join(os.path.join(CTMparaDict["RUN_DIR"], "CTM"), "ctm_pwr.png")
#utilities.viewImg(ctmImgPath)

In [ ]:
### Setup DIE thermal simulation inputs ###
ETCaseName = widgets.Text(value="DEMO", description="ET Project Name", disabled=False)
DIE_H = widgets.FloatText(value=0.0, description="DIE height", disabled=False)
TTFfc = FileChooser("./")
TFDIE = widgets.Text(value="1P10M", description="TTF DIE Label", disabled=False)
totalPWR = widgets.FloatText(value=1.0, description="Total PWR", disabled=False)
deltaT = widgets.FloatText(value=10.0, description="delta T", disabled=False)
ambT = widgets.FloatText(value=25.0, description="Ambient T", disabled=False)
TOPBC = widgets.FloatText(value=1000.0, description="Top DIE BC", disabled=False)
BOTBC = widgets.FloatText(value=1000.0, description="Bot DIE BC", disabled=False)
HB1 = HBox([DIE_H, Label("um")])
HB2 = HBox([Label("Select Thermal TF"), TTFfc, TFDIE])
HB3 = HBox([totalPWR, Label("mW"), deltaT])
HB4 = HBox([ambT, TOPBC, Label("H"), BOTBC, Label("H")])
VBox([ETCaseName, HB1, HB2, HB3, HB4])

In [ ]:
### simple HTC estimation: to refine the HTC values ###
refHTC = utilities.simpleCalHTC(totalPWR.value, designArea, deltaT=deltaT.value)
print("Estimated HTC: {}".format(refHTC))

In [ ]:
### Create ET script ###
DIEparaDict = {"RUN_DIR":caseFolder, "NAME":ETCaseName.value, "CTM":CTMpath, "DIE_LENGTH":DIE_X, \
               "DIE_WIDTH":DIE_Y, "DIE_HEIGHT":str(DIE_H.value), "TTF":TTFfc.selected, "TFDIE":TFDIE.value, \
               "AMBIENT_T":str(ambT.value), "TOPBC":str(TOPBC.value), "BOTBC":str(BOTBC.value) }

templateScript = os.path.join(os.path.join(os.getcwd(), "ET_scripts"), "signalDIE.tcl")
ETscript = utilities.createETscript(DIEparaDict, template=templateScript, outputFolder=caseDB, outputName="DEMO_et.tcl")

ETFolder = os.path.join(caseFolder, ETCaseName.value+"_et")
runcmd = ["#!/bin/csh", "rm -rf {}".format(ETFolder),\
          "source /nfs/site/disks/ifs_itip_disk001/cchuang/Tools/licenses/ansys.csh", \
          "redhawk_sc_et -3dic -ng {}".format(ETscript)]
runcshPath = os.path.join(caseDB, "runET.csh")

progBar = widgets.IntProgress(max=100)
display(progBar)
TProfile = os.path.join(ETFolder, "ThermalProfile_DIE.txt")
checkFiles = [os.path.join(ETFolder, "analysis/STATIC_RUN/thermal/STATIC_RUN.dbg"), \
              TProfile]
utilities.submitJob(runcmd, runcshPath=runcshPath, progressBar=progBar, checkFiles=checkFiles)

In [ ]:
### analysis ET Thermal Profile ###
ETview = RHSCETparser.RHSCETView(TProfile)
ETview.plot(isShow=True)

## UT1. Open RHSC-ET ##
Create a folder "OpenET" for execution

In [ ]:
### Optional: Submit job to open RHSC_ET GUI mode ###
import os

root_ut1 = os.path.join(os.getcwd(), "CASEs")
_caseFolder = utilities.createCASE(root_ut1, "OpenET")
_caseDB = os.path.join(_caseFolder, "DB")

runcmd = ["#!/bin/csh",
          "source /nfs/site/disks/ifs_itip_disk001/cchuang/Tools/licenses/ansys.csh", \
          "redhawk_sc_et -3dic"]
runcshPath = os.path.join(_caseDB, "runETGUI.csh")
utilities.submitJob(runcmd, runcshPath=runcshPath)